<a href="https://colab.research.google.com/github/hkaur103/COMP-593/blob/main/WEEK_4_ADVANCED_SQL_AND_FILES_Harmanjot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4 - Advanced SQL / Writing Files

In this week's lab, we are going to expand our Really Simple Social Network by adding another table that will record connections between people! Because after all, what good is a social network if you can't ... be social?

Afterwards, we will be in a good position to run a report on the data, and then we can sell it to marketers for a massive profit! Isn't it great how few people read the terms of service? `/s`

## Importing our database
Since we are using a new colab notebook, we will need to reimport our database. Check your D2L Submission if you didn't keep a copy, you should be able to redownload it there. Run the below code block and the file upload picker will appear, after uploading your Database file, check that it appears in the `Files` tab of colab.

Remember that you may need to update the listings before it will appear.

In [1]:
from google.colab import files
uploaded = files.upload()


Saving harmanDB.db to harmanDB.db


## Creating our relationship table

In the code block below, we will be creating a new table in our database for `relationships` with the following parameters:


1.   A primary key column called `id` with the `INTEGER` datatype that is `NOT NULL`
2.   A foreign key column called `user_id` that links to the `id` column of the `people` table with the `INTEGER` datatype that is `NOT NULL`
3.   Another foreign key column called `friend_id` that also links to the `id` column of the `people` table with the `INTEGER` datatype that is `NOT NULL`
4.   A `date_created` table with the `DATE` datatype that is `NOT NULL`

The Below codeblock will get you started:



In [2]:
import sqlite3

#Retreive the Connection object, rename the file to match your uploaded DB if necessary.
myConnection = sqlite3.connect('harmanDB.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
myCursor = myConnection.cursor()

#The Primary Key column has been provided for you,
#Replace '#!...' with the remaining column names, datatypes, and foreign key declarations.
#Use the Lecture notes and the previous lab if you are unsure.
createRelationshipsTable = """CREATE TABLE IF NOT EXISTS relationships (
                          id integer PRIMARY KEY NOT NULL,
                          user_id integer NOT NULL,
                          friend_id integer NOT NULL,
                          FOREIGN KEY (user_id) REFERENCES people (id),
                          FOREIGN KEY (friend_id) REFERENCES people (id)
                        );"""

#Once again, we will execute the query, persist the changes, and close our connection.
myCursor.execute(createRelationshipsTable)
myConnection.commit()
myConnection.close()

In [3]:
#Retreive the Connection object, rename the file to match your uploaded DB if necessary.
myConnection = sqlite3.connect('harmanDB.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
myCursor = myConnection.cursor()

#pragma_table_info is an internal SQLite function that will retun information about a table
myCursor.execute("SELECT group_concat(name, ', ') FROM pragma_table_info('relationships')")
print(myCursor.fetchone())

myConnection.close()

('id, user_id, friend_id',)


## Populating Relationship Data

We're not going to be picky about who knows who - so for the purposes of this lab, we're going to use our own version of an AI matchmaker, the `randrange()` method.

Complete the below codeblock, inserting 1000 new rows into the relationships table.
Use a random number between `1 - 1000` for the `user_id` and `friend_id` columns. This will link up to the `id` column of the `people` table and effectivly simulate a random network of relationships.

The below code block is purposely sparse, as you should be able to use the patterns you've identified in the previous lab to complete this task.

In [4]:
import sqlite3
from datetime import datetime #For generating dates and times
from random import randrange #For generating random numbers within a range

#Retreive the Connection object, rename the file to match your uploaded DB if necessary.
myConnection = sqlite3.connect('harmanDB.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
myCursor = myConnection.cursor()

#Complete the below query
relationshipsQuery = """INSERT INTO relationships (user_id, friend_id)
                        VALUES (?, ?);"""

for _ in range(1000):
  #Populate the arguments that will be used in the query
   user_id = randrange(1, 1000)

   friend_id = user_id
   while friend_id == user_id:
     friend_id = randrange(1, 1000)

   args = (user_id, friend_id)
  
  #Execute the query
   myCursor.execute(relationshipsQuery, args)

#Commit your changes and close the connection.
myConnection.commit()
myConnection.close()


In [5]:

import sqlite3
import pprint
#Retreiving the Connection object
myConnection = sqlite3.connect('harmanDB.db')

#Connection object to generate a Cursor object used to run SQL Queries
myCursor = myConnection.cursor()

# statement for output the name of no more than 20 people with a heatmap greater than 1500
selectStatement = """SELECT * FROM relationships
                     LIMIT 20
                     ;"""
 
# executing statement for output
myCursor.execute(selectStatement)
output = myCursor.fetchall()
pprint.pprint(output)

myConnection.close()


[(1, 699, 498),
 (2, 763, 334),
 (3, 359, 620),
 (4, 449, 38),
 (5, 908, 154),
 (6, 52, 246),
 (7, 640, 752),
 (8, 64, 550),
 (9, 532, 460),
 (10, 947, 224),
 (11, 861, 24),
 (12, 857, 594),
 (13, 758, 530),
 (14, 345, 75),
 (15, 228, 678),
 (16, 421, 486),
 (17, 882, 428),
 (18, 178, 244),
 (19, 542, 228),
 (20, 246, 262)]


## Building our Report

Now that our AI Matchmaker has generated our network of relationships, a marketing firm is offering us boatloads of cash for information about the amount of friends the users of our social network have.

The request is to return the name of each user, and the total number of relationships they have made.

If this was explained in awkward pseudo code, it might go something like this:
```
SELECT all relationships
JOIN them to the people they belong to
GROUP the results by the person's ID
return the person's name and COUNT of their relationships
```

After you have collected the data, you will need to write it to a `.txt` file. 

**Your submission will include the report `.txt` file, the copy of your updated `social_network.db` file, and a link to the colab notebook.**

Look at the aggregate functions from Week 3 and the Join examples from Week 4. There are a few different ways that you can structure a query to reach the correct answer.

**At a minimum, you will need to use a `LEFT JOIN` statement, a `GROUP BY` statement, and a `COUNT()` aggregate function.**

The below code block will get you started:

In [9]:
import sqlite3
from pprint import pprint
#Retreive the Connection object, rename the file to match your uploaded DB if necessary.
myConnection = sqlite3.connect('harmanDB.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
myCursor = myConnection.cursor()

#Uncomment the below line after you have determined what the correct mode will be for your file pointer.
#fp = open('report.txt', mode='')

#Complete the below query
query =  """SELECT people.name, COUNT(relationships.id)FROM relationships
            JOIN people ON relationships.user_id = people.id
            GROUP BY relationships.user_id
         """

myCursor.execute(query)
results = myCursor.fetchall()
pprint(results)
myConnection.close() 

#for result in results:
  #You will want to write the results to the file pointer here.
  #Use the newline character \n to create a line break in each write() statement.
  #Use print() statements as sanity checks if you are unsure of the integrity of your data.

#Don't forget to close the file pointer after you have finished!

[('Michelle Rios', 1),
 ('Samuel Hoover', 2),
 ('Nicole Simmons', 3),
 ('Veronica Huerta', 3),
 ('Allison Ortega', 1),
 ('Cheryl Conrad', 3),
 ('Paul Jordan', 1),
 ('Joseph Perez', 2),
 ('Pamela Bridges', 2),
 ('Kevin Kennedy Jr.', 1),
 ('Lisa Contreras', 2),
 ('Ann Ramsey', 1),
 ('Taylor Jenkins', 1),
 ('Madison Morgan', 2),
 ('Tracy Walker', 1),
 ('Kenneth Cole', 1),
 ('Virginia Ortiz', 1),
 ('Amber Mcknight', 1),
 ('Tracy Shaffer', 2),
 ('Bryan Roberts', 2),
 ('Thomas Macias', 2),
 ('Arthur Snow', 1),
 ('Jason Lucero', 1),
 ('Sandra Reed', 2),
 ('Erica Thompson', 1),
 ('Michelle Allen', 1),
 ('Laura Lloyd', 1),
 ('Bethany Williams', 1),
 ('Matthew Morgan', 1),
 ('Dennis Morgan', 1),
 ('Erik Thomas', 2),
 ('Molly Moore', 3),
 ('Gina Harvey', 1),
 ('Elizabeth French', 2),
 ('Joseph Rodriguez', 3),
 ('John Thomas', 2),
 ('Destiny Turner', 2),
 ('Christopher Lewis', 2),
 ('Wayne Lawrence', 3),
 ('Melissa Aguilar', 4),
 ('Veronica Ruiz', 1),
 ('Jennifer Miller', 1),
 ('Jennifer Lynch', 1

### Challenge: Write a CSV file

**This challenge isn't for extra marks, it's an opportunity for you to engage in some self guided discovery, only if you feel like it, and only if you have time.**

If you want to challenge yourself, alter the report so that it conforms with the `.csv` file type. There are a number of ways to do this, you can use the `Pandas` module, the `csv` module, or manually.